In [ ]:
#Updated on 6/28
#All monitoring decisions are included in the MP
#The coefficients of each arc in the MP is exact
#The path with the least detection probability = most probability of being undetected is approximate
using JuMP
using Gurobi
using LightGraphs
const gurobi_env = Gurobi.Env()
function solveMP()

    println("Done loading packages")
    # const MOI = Gurobi.MOI
    
    # setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0) #,LazyConstraints=1)
    println("Done setting param")
    # outfile = "C:/Users/din/Documents/GitHub/UncertainTarget/Log_Test_Output.csv"
    #     io = open(outfile, "w")
    # println(io,"Instance \tPath \tExact \tApprox")
    # global p_exact = []
    # global p_approx = []


    ins = 0 #:10

    include("./Ins"*string(ins)*".jl")
    include("./functionEnumFeasX.jl")
    include("./functionFindQ.jl")
    include("./functionGetCost_SP.jl")

    include("./functionShortestPathBellmanFord.jl")
    include("./functionGenMonitoring.jl")
    include("./functionLP_findM.jl")
    numArcs = length(arcs[:,1])
    X_feas = EnumX(arcs, b_x, numArcs, d_x)
    EnumY = EnumX(arcs, b_y, numArcs, d_y)
    println("EnumY = ", EnumY)
    M_set = [EnumY[1]] #EnumX(arcs, b_y, numArcs, d_y)
    numY = 1 
    R = 10^6
    P_set = [P[1]]

    # global x_now, current_Obj, y_now
    println("X_feas = ", X_feas)
    println("Starting M_set ", M_set, ", P_set = ", P_set)
    Y_sol = []

    numPaths = 1
    newP = true
    newM = true
    iter = 0
    tol = 10^(-6)
    theta_now = -1e12
    inner_Obj = 1-1e12 #set this to the smallest possible value of the obj val
    MP = Model(() -> Gurobi.Optimizer(gurobi_env))
    
    #Heuristics=0.0, Cuts = 0, OutputFlag = 0

    #MASTER PROBLEM
    # MP = direct_model(Gurobi.Optimizer(gurobi_env))
    @variable(MP, x[1:numArcs], Bin)
    @variable(MP, theta >= -1e12)
    @objective(MP, Min, theta)
    @constraint(MP, sum(d_x[a]*x[a] for a=1:numArcs) <= b_x)

    println(MP)
    # while outer_Obj < inner_Obj - tol #newP == true || newM == true
    #     global numArcs, arcs, d_x, q, b_x, origin, destination, R, d_y, b_y
    #     global newP, newM, P_set, M_set, numPaths,numY, iter, outer_Obj, inner_Obj
    println(iter)
    #     cb_calls = Int32[]
    #     cb_calls = Cint[]
    function my_callback_function(cb_data) #, cb_where::Cint)
    #     function my_callback_function(cb_data::Gurobi.CallbackData, cb_where::Int32)
#         global numArcs, arcs, d_x, q, b_x, origin, destination, R, d_y, b_y
#         global P_set, M_set, numPaths,numY, iter #, theta_now, inner_Obj
    #         global iter, P_set, M_set#, lambda, pi_, inner_Obj
    #         global iter_num

#         global iter
        iter += 1
        println("\nIteration number = ", iter)
        println(MP)
        x_now = callback_value.(Ref(cb_data), x)
        theta_now = callback_value.(Ref(cb_data), theta)

        println("\tx_now = ", x_now)
        println("\ttheta_now = ", theta_now)
        newP = false
        numPaths = length(P_set)
        numY = length(M_set)

        println("\tSolving SubProblem")
    #     println("\tPre-solve:P_set = ", P_set, ", M_set = ", M_set)
        P_set, M_set, lambda, pi_, inner_Obj = solveLP_findM(x_now)
        println("\tPost-solve:P_set = ", P_set, ", M_set = ", M_set)
        println("\tlambda = ", lambda, "; pi_ = ", pi_)

        numPaths = length(P_set)
        numY = length(M_set)
    #         dual_Obj = sum(lambda[i]*(-R)*sum(x_now[a] for a in P_set[i]) for i=1:numPaths)+pi_
    #         println("dual ", dual_Obj)
        TOL = 1e-6
        println("theta_now = ", theta_now, " vs subProb = ", inner_Obj)
        if theta_now + TOL >= inner_Obj  #fs_x_current ≈ fm_current # we are done
            @info("No additional constraint from the subproblem")
        end

        if theta_now + TOL < inner_Obj 
            println("Adding constraint")
            con = @build_constraint(theta - sum(lambda[i]*(-R)*sum(x[a] for a in P_set[i]) for i=1:numPaths) >= pi_)
            println(" ", con)
    #         display(con)
            MOI.submit(MP, MOI.LazyConstraint(cb_data),con)
        end
        if iter > 3
            # You can terminate the callback as follows:
            GRBterminate(backend(MP))
        end
    end

    MOI.set(MP, MOI.RawParameter("LazyConstraints"), 1)
#     MOI.set(MP, Gurobi.CallbackFunction(), my_callback_function)
    MOI.set(
        MP,
        MOI.LazyConstraintCallback(),
        my_callback_function,
    )

    optimize!(MP)
end

solveMP()


In [17]:
using JuMP, Gurobi, Test, Combinatorics, LightGraphs

# function solveMP()
    ins = 1 #:10
    include("./Ins"*string(ins)*".jl")
#     global numArcs, arcs, d_x, q, b_x, origin, destination, R, d_y, b_y
#     global newP, newM, P_set, M_set, numPaths,numY
    
    include("./functionEnumFeasX.jl")
    include("./functionFindQ.jl")
    include("./functionGetCost_SP.jl")

    include("./functionShortestPathBellmanFord.jl")
    include("./functionGenMonitoring.jl")
    include("./functionLP_findM.jl")
    
    global numArcs = length(arcs[:,1])
    global X_feas = EnumX(arcs, b_x, numArcs, d_x)
    global EnumY = EnumX(arcs, b_y, numArcs, d_y)
    global M_set = [EnumY[1]] #EnumX(arcs, b_y, numArcs, d_y)
    global numY = 1 
    global R = 10^6#0^6
    global P_set = [P[1]]

#     global x_now, current_Obj, y_now
    println("X_feas = ", X_feas)
    println("Starting M_set ", M_set, ", P_set = ", P_set)
    global Y_sol = []

    global numPaths = 1
    global newP = true
    global newM = true
    global iter = 0
    global tol = 10^(-6)
#     global theta_now = -1e12
#     global inner_Obj = 1-1e12
    
    model = direct_model(Gurobi.Optimizer())
    set_optimizer_attribute(model, "Presolve", 0)
    set_optimizer_attribute(model, "Heuristics", 0)
# Presolve = 0,
#         PreCrush = 1,
#         Heuristics = 0,

    @variable(model, x[1:numArcs], Bin)
    @variable(model, theta >= -1e12)
    @objective(model, Min, theta)
    @constraint(model, sum(d_x[a]*x[a] for a=1:numArcs) <= b_x)
    
#     @variable(model, 0 <= x <= 2.5, Int)
#     @variable(model, 0 <= y <= 2.5, Int)
#     @objective(model, Max, y)
#     println("\nM_set = ", M_set," \t P_set = ", P_set)
    cb_calls = Cint[]
    function my_callback_function(cb_data, cb_where::Cint)
    global numArcs, arcs, d_x, q, b_x, origin, destination, R, d_y, b_y
    global newP, newM, P_set, M_set, numPaths,numY, iter, theta_now, inner_Obj
        # You can reference variables outside the function as normal
        push!(cb_calls, cb_where)
        iter += 1
#         println("\niter = ", iter)
#         println("\nM_set = ", M_set," \t P_set = ", P_set)
        # You can select where the callback is run
        if cb_where != GRB_CB_MIPSOL && cb_where != GRB_CB_MIPNODE
            return
        end
        # You can query a callback attribute using GRBcbget
        if cb_where == GRB_CB_MIPNODE
            resultP = Ref{Cint}()
            GRBcbget(cb_data, cb_where, GRB_CB_MIPNODE_STATUS, resultP)
            if resultP[] != GRB_OPTIMAL
                return  # Solution is something other than optimal.
            end
        end
        # Before querying `callback_value`, you must call:
        Gurobi.load_callback_variable_primal(cb_data, cb_where)
        x_val = callback_value.(Ref(cb_data), x)
        theta_val = callback_value(cb_data, theta)
        P_set, M_set, lambda, pi_, inner_Obj = solveLP_findM(x_val)
        numPaths = length(P_set)
        numY = length(M_set)
#         println("x_val = ", x_val)
#         println("lambda = ", lambda, "\t pi_ = ", pi_)
#         println("\nM_set = ", M_set," \t P_set = ", P_set)
        # You can submit solver-independent MathOptInterface attributes such as
        # lazy constraints, user-cuts, and heuristic solutions.
#         if y_val - x_val > 1 + 1e-6
#             con = @build_constraint(y - x <= 1)
#             MOI.submit(model, MOI.LazyConstraint(cb_data), con)
#         elseif y_val + x_val > 3 + 1e-6
#             con = @build_constraint(y + x <= 3)
#             MOI.submit(model, MOI.LazyConstraint(cb_data), con)
#         end
#         lambda = [0.30103, 0.69897]
#         pi_ = 0.7895890626254753
        TOL = 10^(-12)
        
        if theta_val + TOL < inner_Obj 
        println("\niter = ", iter)
        println("theta_val = ", theta_val, "\tinner_Obj = ", inner_Obj)
#             if x_val == [0,0,0]
                println("x_val = ", x_val)
                println("lambda = ", lambda, "\t pi_ = ", pi_)
                con = @build_constraint(theta >= sum(lambda[i]*(-R)*sum(x[a] for a in P_set[i]) for i=1:numPaths) + pi_)
                println(sum(lambda[i]*(-R)*sum(x[a] for a in P_set[i]) for i=1:numPaths) + pi_)
                MOI.submit(model, MOI.LazyConstraint(cb_data),con)
#             end
#             if x_val == [1,0,0]
#                 println("x_val = ", x_val)
#                 println("lambda = ", lambda, "\t pi_ = ", pi_)
#                 con = @build_constraint(theta >= sum(lambda[i]*(-R)*sum(x[a] for a in P_set[i]) for i=1:numPaths) + pi_)
#                 println(sum(lambda[i]*(-R)*sum(x[a] for a in P_set[i]) for i=1:numPaths) + pi_)
#                 MOI.submit(model, MOI.LazyConstraint(cb_data),con)
#             end
#             if x_val == [0,0,1]
#                 println("x_val = ", x_val)
#                 println("lambda = ", lambda, "\t pi_ = ", pi_)
#                 con = @build_constraint(theta >= sum(lambda[i]*(-R)*sum(x[a] for a in P_set[i]) for i=1:numPaths) + pi_)
#                 println(sum(lambda[i]*(-R)*sum(x[a] for a in P_set[i]) for i=1:numPaths) + pi_)
#                 MOI.submit(model, MOI.LazyConstraint(cb_data),con)
#             end
#             println("Adding constraint")
#             con = @build_constraint(theta - sum(lambda[i]*(-R)*sum(x[a] for a in P_set[i]) for i=1:numPaths) >= pi_)
#             println(" ", con)
    #         display(con)
#             MOI.submit(model, MOI.LazyConstraint(cb_data),con)
        end
#         if rand() < 0.1
#             # You can terminate the callback as follows:
#             GRBterminate(backend(model))
#         end
        return
    end
    # You _must_ set this parameter if using lazy constraints.
    MOI.set(model, MOI.RawParameter("LazyConstraints"), 1)
    MOI.set(model, Gurobi.CallbackFunction(), my_callback_function)
    println(model)
    optimize!(model)
    println(model)
    println("x = ", JuMP.value.(x))
    println("theta = ", JuMP.value.(theta))
# end
# solveMP()

X_feas = Any[[2, 6, 7], [6, 7, 8], [6, 7, 9], [1, 3, 5, 6, 7]]
Starting M_set Array{Int64,1}[[2]], P_set = Array{Int64,1}[[1, 8]]
Min theta
Subject to
 x[1] binary
 x[2] binary
 x[3] binary
 x[4] binary
 x[5] binary
 x[6] binary
 x[7] binary
 x[8] binary
 x[9] binary
 theta >= -1.0e12
 2 x[1] + 5 x[2] + 2 x[3] + 7 x[4] + x[5] + 5 x[8] + 5 x[9] <= 5.0

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1 rows, 10 columns and 7 nonzeros
Model fingerprint: 0x7157c3dc
Variable types: 1 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+12, 1e+12]
  RHS range        [5e+00, 5e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Variable types: 1 continuous, 9 integer (9 binary)

Root relaxation: objective -1.000000e+12, 0 iterations, 0.